<a href="https://colab.research.google.com/github/daianjibetu/TMDS_TEMPEST_Simulator/blob/main/TMDS_TEMPEST_Simulator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

2進数に変換

In [ ]:
def IntToBits(indata):
    outdata = []
    
    for element in indata:
        str_array = list(format(element, '08b'))
        outdata.append([int(s) for s in str_array])
    
    return outdata

ビット順序反転

In [ ]:
def BitsReverse(indata):
    outdata = []

    for elements in indata:
        outdata.append(list(reversed(elements)))

    return outdata

XNORプロセス

In [ ]:
def XNOR(indata, isEncode):
    outdata = []

    outdata.append(indata[0])
    for i in range(1,8):
        if isEncode:
            if (outdata[i - 1] == indata[i]):
                outdata.append(1)
            else:
                outdata.append(0)
        else:
            if (indata[i - 1] == indata[i]):
                outdata.append(1)
            else:
                outdata.append(0)
    
    if isEncode:
        outdata.append(0)

    return outdata

XORプロセス

In [ ]:
def XOR(indata, isEncode):
    outdata = []

    outdata.append(indata[0])
    for i in range(1,8):
        if isEncode:
            if (outdata[i - 1] == indata[i]):
                outdata.append(0)
            else:
                outdata.append(1)
        else:
            if (indata[i - 1] == indata[i]):
                outdata.append(0)
            else:
                outdata.append(1)
    
    if isEncode:
        outdata.append(1)

    return outdata

Transition Minimized（遷移時間最短）

In [ ]:
def TransitionMinimized(indata):
    outdata = []

    for elements in indata:
        ones = sum(elements)
        if (ones > 4 or (ones == 4 and elements[0] == 0)):
            outdata.append(XNOR(elements, True))
        else:
            outdata.append(XOR(elements, True))
    
    return outdata

ビット反転

In [ ]:
def inv(bit):
    if(bit == 0):   
        return 1
    else:
        return 0

Throughプロセス

In [ ]:
def Through(indata, isEncode):
    outdata = []

    for i in range(8):
        outdata.append(indata[i])
    
    outdata.append(indata[8])
    
    if isEncode:
        outdata.append(0)

    return outdata

Inverseプロセス

In [ ]:
def Inverse(indata, isEncode):
    outdata = []

    for i in range(8):
        outdata.append(inv(indata[i]))
    
    outdata.append(indata[8])
    
    if isEncode:
        outdata.append(1)

    return outdata

DC-Balancing（DCバランス）

In [ ]:
def DCBalancing(indata):
    outdata = []

    cnt = 0

    for elements in indata:
        zeros = 0
        ones = 0
        for i in range(8):
            ones += elements[i]
        zeros = 8 - ones
        
        if (cnt == 0 or zeros == ones):
            if (elements[8] == 0):
                outdata.append(Inverse(elements, True))
                cnt += (zeros - ones)
            else:
                outdata.append(Through(elements, True))
                cnt += (ones - zeros)
        else:
            if ((cnt > 0 and ones > zeros) or (cnt < 0 and zeros > ones)):
                outdata.append(Inverse(elements, True))
                cnt += ((zeros - ones) + 2 * elements[8])
            else:
                outdata.append(Through(elements, True))
                cnt += ((ones - zeros) - 2 * inv(elements[8]))
    
    return outdata

エンコード

In [ ]:
def TMDSEncode(indata):
    tmpdata = TransitionMinimized(indata)
    outdata = DCBalancing(tmpdata)

    return outdata

実行用

In [ ]:
import numpy as np

from PIL import Image
from google.colab import files
from tqdm.notebook import tqdm

画像アップロード

In [ ]:
file = files.upload()
file_name = list(file.keys())[0]
#file_name = ""
img = Image.open(file_name)

グレースケールに変換

In [ ]:
gray_img = img.convert("L")

画像からNumPy配列に

In [ ]:
img_array = np.array(gray_img)

print(img_array.shape)

エンコード処理

In [ ]:
outdata = []

for elements in tqdm(img_array):

    array_1 = IntToBits(elements.tolist())
    array_2 = BitsReverse(array_1)
    array_3 = TMDSEncode(array_2)

    outdata.append(array_3)

メイン処理

In [ ]:
for i, row in enumerate(tqdm(outdata)):
    for j, col in enumerate(row):
        if col[-1]:
            img_array[i][j] = ~img_array[i][j] & 255

NumPy配列から画像に

In [ ]:
new_img = Image.fromarray(np.uint8(img_array))

画像のダウンロード

In [ ]:
new_img.save("new_img.png")
files.download("new_img.png")